# AOC2022

## Day 15 / Part 2 / Beacon Exclusion Zone

Problem Description: https://adventofcode.com/2022/day/15#part2

Input: [Example](aoc2022_day15_example.txt)

### Notes
- The derivation of intersection points between a [diamond](https://mathworld.wolfram.com/Diamond.html) at $(center_y, center_x)$ with radius $r$ and a horizonzal line at $y$ is described in [part 1](aoc2022_day15_1.ipynb)

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 15, part 2."""
import logging
import re
import sys

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day15_example.txt"

In [3]:
MAX_ROW = 20

In [4]:
class SensorField:
    """
    Diamond scanning field of a sensor which contains only a single beacon.
    """
    pos = None
    radius = None

    def __init__(self, pos, radius):
        self.pos = pos
        self.radius = radius

    def does_intersect(self, row):
        """Check if a row intersects the field."""
        return manhattan_dist(self.pos, [row, self.pos[1]]) <= self.radius

    def intersect(self, row):
        """
        Calculate the (min_x, max_x, range) of the intersection between
        a row and the field.
        """
        min_x = -self.radius + abs(row - self.pos[0]) + self.pos[1]
        max_x = self.radius - abs(row - self.pos[0]) + self.pos[1]
        return min_x, max_x, max_x - min_x

In [5]:
def manhattan_dist(point1, point2):
    """Calculate manhattan distance between point1 and point2."""
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])


def find_distress_beacon(sensor_fields):
    """Calculate the position of the distress beacon."""
    try:
        LOGGER.debug("search destress beacon...")

        for row in range(MAX_ROW):
            row_intersections = []
            for sensor_field in sensor_fields:
                if sensor_field.does_intersect(row):
                    inter_min_x, inter_max_x, _ = sensor_field.intersect(row)
                    row_intersections.append([inter_min_x, inter_max_x])

            row_intersections = list(sorted(row_intersections))

            x_pos = -1
            for inter_min_x, inter_max_x in row_intersections:
                if x_pos + 1 < inter_min_x:
                    raise StopIteration([row, x_pos + 1])
                x_pos = max(x_pos, inter_max_x)

            if x_pos < MAX_ROW:
                raise StopIteration([row, MAX_ROW])

            LOGGER.debug("  %s%%", row / MAX_ROW * 100)
    except StopIteration as ex:
        LOGGER.debug("  found beacon at %s", ex.args[0])
        return ex.args[0]

    return None

In [6]:
def main():
    """Main function to solve puzzle."""
    sensor_fields = []
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        LOGGER.debug("read input...")
        for line in [line.rstrip() for line in file_obj.readlines()]:
            (
                sensor_x_pos, sensor_y_pos,
                beacon_x_pos, beacon_y_pos
            ) = map(int, re.search(
                r"Sensor at x=([-]?\d+), y=([-]?\d+): " +
                r"closest beacon is at x=([-]?\d+), y=([-]?\d+)",
                line
            ).groups())

            sensor_pos = (sensor_y_pos, sensor_x_pos)
            beacon_pos = (beacon_y_pos, beacon_x_pos)

            LOGGER.debug(
                "  found sensor %s with closest beacon at %s",
                sensor_pos, beacon_pos
            )

            sensor_fields.append(
                SensorField(sensor_pos, manhattan_dist(sensor_pos, beacon_pos))
            )

    LOGGER.debug("")

    distress_beacon_pos = find_distress_beacon(sensor_fields)

    LOGGER.debug("")

    print(
        f"solution: {distress_beacon_pos[1]*4000000 + distress_beacon_pos[0]}"
    )

In [7]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 56000011
